# Ray Core: Remote Functions as Tasks

© 2022, Anyscale Inc. All Rights Reserved.

## Introduction
Ray enables arbitrary functions to be executed asynchronously on separate Python workers. These asynchronous Ray functions are called “tasks.” You can specify task's resource requirements in terms of CPUs, GPUs, and custom resources. These resource requests are used by the cluster scheduler to distribute tasks across the cluster for parallelized execution.  

<img src="../_static/assets/Ray_Core/Ray_Core_1_Remote_Functions/python_to_ray_concept_map.png" height="55%" width="50%">

### Learning objectives
In this this tutorial, we learn about:
 * Remote Task Parallel Pattern
 * Stateless remote functions as distributed tasks
 * Serial vs Parallel execution 
 * Understand the concept of a Ray task 
 * Easy API to convert an existing Python function into a Ray remote task
 * Walk through a map-reduce and distribute batch inference use cases

In [ ]:
import os
import time
import logging

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from numpy import loadtxt
import ray

## Tasks Parallel Pattern

Ray converts decorated functions with `@ray.remote` into stateless tasks, scheduled anywhere on a Ray node's worker in the cluster. 

Where they will be executed (and by whom), you don't have to worry about its details. All that is taken care for you. Nor do 
you have to reason about it — all that burden is Ray's job. You simply take your existing Python functions and covert them into 
distributed stateless *Ray Tasks*: **as simple as that!**

### Example 1: Serial vs Parallelism

Let's look at simple tasks running serially and then in parallel. For illustration, we'll use a simple task, but this could be a compute-intensive task as part of your workload.


There are a few key differences between the original function and the decorated one:

**Invocation**: The regular version is called with `regular_function()`, whereas the remote version is called with `remote_function.remote()`. Keep this pattern in mind for all Ray remote execution methods.

**Mode of execution and return values**: `regular_function` executes synchronously and returns the result of the function as the value `1` (in our case), whereas `remote_function` immediately returns an `ObjectID` (a future) and then executes the task in the background on a remote worker process. The result of the future is obtained by calling `ray.get` on the `ObjectID`. This is a blocking function.

In [ ]:
# A regular Python function.
def regular_function():
    time.sleep(1)
    return 1

In [ ]:
# A Ray remote function.
@ray.remote
def remote_function():
    time.sleep(1)
    return 1

Let's launch a Ray cluster on our local machine.

In [ ]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

In [ ]:
# Let's invoke the regular function
assert regular_function() == 1

In [ ]:
# Let's invoke the remote function.
obj_ref = remote_function.remote()
obj_ref

In [ ]:
assert ray.get(obj_ref) == 1

#### Serial execution in Python with no parallelism
Invocations of `regular_function` in a comprehension loop happens `serially`:

In [ ]:
# These are executed one at a time, back-to-back, in a list comprehension
results = [regular_function() for _ in range(10)]
assert sum(results) == 10

#### Parallel execution in Python with Ray

Invocations of `remote_function` in a loop happen `asynchronously` and in parallel:

In [ ]:
# Executing these functions, in comprehension list, happens at the same time in the background,
# and we get the results using ray.get.

results = [remote_function.remote() for _ in range(10)]
assert sum(ray.get(results)) == 10

### Example 2: Adding two np arrays

<img src="../_static/assets/Ray_Core/Ray_Core_1_Remote_Functions/remote_task_api_add_array.png" width="50%" height="25%">

Define a function as a Ray task to read an array

In [ ]:
@ray.remote
def read_array(fn: str) -> np.array:
    arr = loadtxt(fn, comments="#", delimiter=",", unpack=False)
    return arr.astype("int")

Define a function as a Ray task to add two np arrays return the sum

In [ ]:
@ray.remote
def add_array(arr1: np.array, arr2: np.array) -> np.array:
    return np.add(arr1, arr1)

Define a function as a Ray task to sum the contents of an np array

In [ ]:
@ray.remote
def sum_array(arr1: np.array) -> int:
    return np.sum(arr1)

Now let's execute our tasks. For now we will run Ray locally on our laptop or on a single node, with potential access to utilize all the available cores when necessary.

Ray executes immediately and returns an object reference `ObjectRef` as a future. This enables Ray to parallelize tasks and execute them asynchronously.

#### Read both arrays

Use the `func_name.remote(args)` extention to invoke a remote Ray Task

In [ ]:
if os.getcwd().endswith("Ray_Core"):
    path_root = os.getcwd()
else:
    path_root = os.path.join(os.getcwd(), "Ray_Core")

In [ ]:
obj_ref_arr1 = read_array.remote(os.path.join(path_root, "data/file_1.txt"))
print(f"array 1: {obj_ref_arr1}")

In [ ]:
obj_ref_arr2 = read_array.remote(os.path.join(path_root, "data/file_2.txt"))
print(f"array 2: {obj_ref_arr2}")

#### Add both arrays

Let's add our two arrays by calling the remote method. *Note*: We are sending Ray `ObjectRef` references as arguments. Those arguments will be resolved inline and fetched from owner's object store. That is, the cluster node that creates the `ObjectRef` owns the meta data associated and stores it in its object store. 

Ray scheduler is aware of where these object references reside or who owns them, so it will schedule this remote task on node on the worker process for data locality.

In [ ]:
result_obj_ref = add_array.remote(obj_ref_arr1, obj_ref_arr2)
result_obj_ref

#### Fetch the result

This will task if not finished will block during `.get(object_ref`)

In [ ]:
result = ray.get(result_obj_ref)
print(f"Result: add arr1 + arr2: \n {result}")

Add the array elements within an `np.array` and get the sum. 
**Note** that we are sending `ObjRefs` as arguments to the function. Ray will resolve or fetch the value of these arrays. 

In [ ]:
sum_1 = ray.get(sum_array.remote(obj_ref_arr1))
sum_2 = ray.get(sum_array.remote(obj_ref_arr2))

In [ ]:
print(f"Sum of arr1: {sum_1}")
print(f"Sum of arr2: {sum_2}")

#### Any questions?

### Example 3: Generating Fibonnaci series

Let's define two functions: one runs locally or serially, the other runs on a Ray cluster (local or remote). This example is borrowed and refactored from our 
blog: [Writing your First Distributed Python Application with Ray](https://www.anyscale.com/blog/writing-your-first-distributed-python-application-with-ray). 
(This is an excellent tutorial to get started with the concept of why and when to use Ray tasks and Ray Actors. Highly recommended read!)

Another similar blog of interest is how to compute the value of **pi**: [How to scale Python multiprocessing to a cluster with one line of code](https://medium.com/distributed-computing-with-ray/how-to-scale-python-multiprocessing-to-a-cluster-with-one-line-of-code-d19f242f60ff).

In [ ]:
# Function for local execution
def generate_fibonacci(sequence_size):
    fibonacci = []
    for i in range(0, sequence_size):
        if i < 2:
            fibonacci.append(i)
            continue
        fibonacci.append(fibonacci[i - 1] + fibonacci[i - 2])
    return len(fibonacci)

In [ ]:
# Function for remote Ray task with just a wrapper
@ray.remote
def generate_fibonacci_distributed(sequence_size):
    return generate_fibonacci(sequence_size)

In [ ]:
# Get the number of cores
os.cpu_count()

In [ ]:
# Normal Python in a single process
def run_local(sequence_size):
    results = [generate_fibonacci(sequence_size) for _ in range(os.cpu_count())]
    return results

In [ ]:
%%time
run_local(100000)

In [ ]:
# Distributed on a Ray cluster
def run_remote(sequence_size):
    results = ray.get(
        [
            generate_fibonacci_distributed.remote(sequence_size)
            for _ in range(os.cpu_count())
        ]
    )
    return results

In [ ]:
%%time
run_remote(100000)

#### Any questions?

### Example 4: Use case of `tasks` for map-reduce

The `map-reduce` pattern is a good use case for writing distributed applications with Ray core APIs. For _map_, this example uses Ray tasks to execute a 
given function multiple times in parallel (on a separate process on a node).  

We then use `ray.get`, as part of the `reduce` process, to fetch the results of each of these functions.

<img src="../_static/assets/Ray_Core/Ray_Core_1_Remote_Functions/map_reduce_architecture.svg">

#### Single-threaded map

In [ ]:
items = list(range(100))
map_func = lambda i: i * 2
output = [map_func(i) for i in items]
print(output)

#### Ray parallel map
Use the `@ray.remote` decorator to convert this `map`function into a Ray task. It takes an object and func argument and invokes the function to process the object.
Simple and elegant!

In [ ]:
@ray.remote
def map(obj, f):
    return f(obj)

In [ ]:
items = list(range(100))
map_func = lambda i: i * 2

# map.remote() will return an objRef to the computed value. We fetch
# that value using ray.get
output = ray.get([map.remote(i, map_func) for i in items])
print(output)

#### Single-threaded reduce
For reduce, let's imagine that we want to sum up the numbers computed from
our map function.

In [ ]:
items = list(range(100))
map_func = lambda i: i * 2
output = sum([map_func(i) for i in items])
output

#### Ray parallel map and reduce

In [ ]:
@ray.remote
def map(obj, f):
    return f(obj)


# Our reduce operation is expecting multipe arguments.
# It sums up all arguments using np.sum(elements)
@ray.remote
def sum_results(*elements):
    return np.sum(elements)

Let's do our Ray parallel map. Note that comprehension list is a collection of `ObjRefs`, each element returned by `map.remote(i, map_func)`

In [ ]:
items = list(range(100))
map_func = lambda i: i * 2
remote_elements = [map.remote(i, map_func) for i in items]
remote_elements[:2]

##### Simple reduce
The `sum_results.remote()` as a reduce step returns the `ObjectRef` to results
of all the values in the elements.

In [ ]:
# simple reduce
remote_final_sum = sum_results.remote(*remote_elements)
# fetch the reduce sumed result
result = ray.get(remote_final_sum)
print(result)

##### Tree reduce
Simply break into intermediate results, followed by the final reduce. 
In short break into five groups of 20 object references, and then final
reduce.

In [ ]:
# Tree reduce using comprehension list.
# Split in five groups of 20 ObjecRefs for intermediate reduce,
# followed by final reduce, for all 100 elements
intermediate_results = [
    sum_results.remote(*remote_elements[i * 20 : (i + 1) * 20]) for i in range(5)
]

# Get the reduce results of these groups
remote_final_sum = sum_results.remote(*intermediate_results)
result = ray.get(remote_final_sum)
print(result)

### Example 5:  How to use Tasks for distributed batch inference 

Batch inference is a common distributed application workload in machine learning. It's a process of using a trained model to generate predictions for a collection of observations. 
Primarily, it has the following elements:

**Input dataset**: This is a large collection of observations to generate predictions for. The data is usually stored in an external storage system like S3, HDFS or database, across
many files.
**ML model**: This is a trained ML model that is usually also stored in an external storage system or in a model store.
**Predictions**: These are the outputs when applying the ML model on observations. Normally, predictions are usually written back to the storage system.

For purpose of this tutorial, we make the following provisions:
 * create a dummy model that returns some fake prediction
 * use real-world NYC taxi data to provide large data set for batch inference
 * return the predictions instead of writing it back to the disk

As an example of scaling pattern called Different Data Same Function (DDSF), also known as Distributed Data Parallel (DDP) paradigm, our function in this digaram is the 
pretrained **model** and the data is split and disributed as **shards**.

<img src="../_static/assets/Ray_Core/Ray_Core_1_Remote_Functions/batch_inference_architecture.png" width="25%" height="25%">


Define a Python closure to load our pretrained model. This model is just a fake model that predicts whether a 
tip is warranted continent on the number of fares (2 or more) on collective rides.

**Note**: This prediction is fake. The real model will invoke model's `model.predict(input_data)`. Yet
it suffices for this example.

In [ ]:
def load_trained_model():
    # A fake model that predicts whether tips were given based on number of passengers in the taxi cab.
    def model(batch: pd.DataFrame) -> pd.DataFrame:
        # Some model payload so Ray copies the model in the shared plasma store to tasks scheduled across nodes.
        model.payload = np.arange(100, 100_000_000, dtype=float)
        model.cls = "regression"

        # give a tip if 2 or more passengers
        predict = batch["passenger_count"] >= 2
        return pd.DataFrame({"score": predict})

    return model

Let's define a Ray task that will handle each shard of the NYC taxt data

In [ ]:
@ray.remote
def make_model_batch_predictions(model, shard_path):
    print(f"Batch inference for shard file: {shard_path}")
    df = pq.read_table(shard_path).to_pandas()
    result = model(df)

    # Return our prediction data frame
    return result

Get the 8 files consisting of NYC data per month

In [ ]:
# 12 files, one for each remote task.
input_files = [
    f"s3://anonymous@air-example-data/ursa-labs-taxi-data/downsampled_2009_full_year_data.parquet"
    f"/fe41422b01c04169af2a65a83b753e0f_{i:06d}.parquet"
    for i in range(8)
]

`ray.put()` the model just once to local object store, and then pass the reference to the remote tasks.
This is Ray core API for putting objects into the Ray Plasma store. We discuss these APIs and Plasma store
in the next tutorial. 

In [ ]:
# Get the model
model = load_trained_model()

# Put the model object into the shared object store.
model_ref = ray.put(model)
model_ref

In [ ]:
# List for holding all object references from the model's predictions
result_refs = []

# Launch all prediction tasks. For each file create a Ray remote task
# to do a batch inference
for file in input_files:

    # Launch a prediction task by passing model reference and shard file to it.
    # NOTE: it would be highly inefficient if you are passing the model itself
    # like  make_model_prediction.remote(model, file), which in order to pass the model
    # to remote node will ray.put(model) for each task, potentially overwhelming
    # the local object store and causing out-of-memory or out-of-disk error.
    result_refs.append(make_model_batch_predictions.remote(model_ref, file))

Fetch the results

In [ ]:
results = ray.get(result_refs)

In [ ]:
# Let's check predictions and output size.
for r in results:
    print(
        f"Predictions dataframe size: {len(r)} | Total score for tips: {r['score'].sum()}"
    )

In [ ]:
ray.shutdown()

## Summary

### Exercises

1. Increase the fibonacci with 200K, 300K
1. Add a compute intensive function; pick some function from your repo and convert to a remote task.
1. (Optional) Run how to [compute PI](extra/highly_parallel.ipynb). **Note**: You can tweak with the `FULL_SAMPLE_COUNT`, to adjust the accuracy to the value of `math.pi`. `100 billion samples` may take too long. Play with this number.

### Homework
1. Try writing a map-reduce app for word or character count in a list. Try first with simple case of a few lines, then extend it to a large file.
1. Try using local [bubble sort](https://www.geeksforgeeks.org/python-program-for-bubble-sort/) and remote bubble sort
1. Do you see the difference for small and large numbers?
1. Read this [blog](https://www.anyscale.com/blog/parallelizing-python-code) and try some examples.
1. Take an existing regression model, save it as in a model format, use this scaling technique to do batch inference at scale and in parallel